In [1]:
# Import Libraries -------------------------------------------------------------------------------------------------------------------------------------------------------------
import os
import h5py
import keras
import loss
import Helper
import allMetrics
import numpy as np
import tensorflow as tf
import UNetModel_3D_simplified
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
ds_num = 1

train_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Scans/resampled_trainDS{ds_num}.hdf5"
train_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Mask_Scans/resampled_trainmaskDS{ds_num}.hdf5"

train_DatasetName = f"ResampledtrainScans_DataSet{ds_num}"
train_maskDatasetName = f"ResampledtrainMaskScans_DataSet{ds_num}"

valid_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Scans/resampled_validDS{ds_num}.hdf5"
valid_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Mask_Scans/resampled_validmaskDS{ds_num}.hdf5"

valid_DatasetName = f"ResampledvalidScans_DataSet{ds_num}"
valid_maskDatasetName = f"ResampledvalidMaskScans_DataSet{ds_num}"

test_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/test_Scans/resampled_testDS{ds_num}.hdf5"
test_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/test_Mask_Scans/resampled_testmaskDS{ds_num}.hdf5"

test_DatasetName = f"ResampledtestScans_DataSet{ds_num}"
test_maskDatasetName = f"ResampledtestMaskScans_DataSet{ds_num}"

In [4]:
with h5py.File(train_fileName, 'r') as hf: # File Dir
    train_array = hf[train_DatasetName][:]
    
with h5py.File(train_maskfileName,'r') as hf:
    train_mask_array = hf[train_maskDatasetName][:]
    
with h5py.File(valid_fileName, 'r') as hf: # File Dir
    valid_array = hf[valid_DatasetName][:]
    
with h5py.File(valid_maskfileName,'r') as hf:
    valid_mask_array = hf[valid_maskDatasetName][:]
    
with h5py.File(test_fileName, 'r') as hf: # File Dir
    test_array = hf[test_DatasetName][:]
    
with h5py.File(test_maskfileName,'r') as hf:
    test_mask_array = hf[test_maskDatasetName][:]

In [5]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)
print('--')
print(test_array.shape)
print(test_mask_array.shape)

(6543, 64, 64, 64)
(6543, 64, 64, 64)
--
(1596, 64, 64, 64)
(1596, 64, 64, 64)
--
(2403, 64, 64, 64)
(2403, 64, 64, 64)


In [6]:
train_scan = np.concatenate((train_array,valid_array,test_array))
train_mask_scan = np.concatenate((train_mask_array,valid_mask_array,test_mask_array))

In [12]:
train_array = np.expand_dims(train_scan, axis=4)
train_mask_array = np.expand_dims(train_mask_scan, axis=4)

valid_array = np.expand_dims(test_array, axis=4)
valid_mask_array = np.expand_dims(test_mask_array, axis=4)

In [13]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)

(10542, 64, 64, 64, 1)
(10542, 64, 64, 64, 1)
--
(2403, 64, 64, 64, 1)
(2403, 64, 64, 64, 1)


In [14]:
LR = 0.001
opt = tf.keras.optimizers.Nadam(LR)

input_shape = (64,64,64,1)
num_class = 1

metrics = [allMetrics.dice_coef]

In [15]:
model = UNetModel_3D_simplified.build_unet(input_shape, n_classes = num_class)

model.compile(optimizer=opt, loss=loss.tversky_crossentropy, metrics=metrics)
print(model.summary())

2023-03-15 13:41:02.759869: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 13:41:03.258025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37831 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


sigmoid
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 64,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 64, 64, 64,   896         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 64,   128        ['conv3d[0][0]']                 
 alization)                     32)                                                   

In [16]:
csv_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/CSVLogs/resampled_5mm.csv'
model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/resampled_5mm.hdf5'

In [17]:
my_callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, mode = 'auto'),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, mode = 'auto'),
    CSVLogger(csv_path, separator=',', append=True),
    ModelCheckpoint(filepath=model_checkpoint_path,
                    monitor='val_loss',
                    mode='auto',
                    verbose=1,
                    save_best_only= True)
]
#     CustomCallBack()

In [ ]:
model_name = 'resampled_5mm (300 Epochs)'
# Helper.telegram_bot_sendtext(f'Model {model_name} started training')

history = model.fit(train_array,
                    train_mask_array,
                    batch_size=5,
                    epochs=300,
                    verbose=1,
                    shuffle = True,
                    validation_data=(valid_array, valid_mask_array),
                    callbacks=my_callbacks)

Epoch 1/300


2023-03-15 13:41:28.749935: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2023-03-15 13:41:29.596574: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


2109/2109 [==============================] - ETA: 0s - loss: 0.7413 - dice_coef: 0.5047
Epoch 00001: val_loss improved from inf to 0.58953, saving model to /media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/resampled_5mm.hdf5
2109/2109 [==============================] - 215s 98ms/step - loss: 0.7413 - dice_coef: 0.5047 - val_loss: 0.5895 - val_dice_coef: 0.5956 - lr: 0.0010
Epoch 2/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.6343 - dice_coef: 0.6219
Epoch 00002: val_loss did not improve from 0.58953
2109/2109 [==============================] - 204s 97ms/step - loss: 0.6343 - dice_coef: 0.6219 - val_loss: 1.3843 - val_dice_coef: 0.0104 - lr: 0.0010
Epoch 3/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.5921 - dice_coef: 0.6635
Epoch 00003: val_loss did not improve from 0.58953
2109/2109 [==============================] - 202s 96ms/step - loss: 0.5922 - dice_coef: 0.6633 - val_loss: 0.7224 - val_dice_coef: 0.4991 - lr: 0.0010
Epoch 4

Epoch 26/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.3714 - dice_coef: 0.7927
Epoch 00026: val_loss did not improve from 0.30333
2109/2109 [==============================] - 202s 96ms/step - loss: 0.3714 - dice_coef: 0.7927 - val_loss: 0.4677 - val_dice_coef: 0.6806 - lr: 0.0010
Epoch 27/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.3634 - dice_coef: 0.7977
Epoch 00027: val_loss did not improve from 0.30333
2109/2109 [==============================] - 202s 96ms/step - loss: 0.3634 - dice_coef: 0.7977 - val_loss: 0.4338 - val_dice_coef: 0.6921 - lr: 0.0010
Epoch 28/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.3654 - dice_coef: 0.7936
Epoch 00028: val_loss did not improve from 0.30333
2109/2109 [==============================] - 203s 96ms/step - loss: 0.3653 - dice_coef: 0.7936 - val_loss: 0.9125 - val_dice_coef: 0.4753 - lr: 0.0010
Epoch 29/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.3364 - dice_coe

Epoch 50/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2777 - dice_coef: 0.8455
Epoch 00050: val_loss did not improve from 0.23881
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2777 - dice_coef: 0.8455 - val_loss: 0.3028 - val_dice_coef: 0.7915 - lr: 5.0000e-04
Epoch 51/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2667 - dice_coef: 0.8492
Epoch 00051: val_loss did not improve from 0.23881
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2667 - dice_coef: 0.8492 - val_loss: 0.2995 - val_dice_coef: 0.7292 - lr: 5.0000e-04
Epoch 52/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2678 - dice_coef: 0.8485
Epoch 00052: val_loss did not improve from 0.23881
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2677 - dice_coef: 0.8485 - val_loss: 0.2720 - val_dice_coef: 0.7937 - lr: 5.0000e-04
Epoch 53/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.266

Epoch 75/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2332 - dice_coef: 0.8626
Epoch 00075: val_loss did not improve from 0.20230
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2331 - dice_coef: 0.8626 - val_loss: 0.2107 - val_dice_coef: 0.8449 - lr: 5.0000e-04
Epoch 76/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2336 - dice_coef: 0.8626
Epoch 00076: val_loss did not improve from 0.20230
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2335 - dice_coef: 0.8626 - val_loss: 0.2125 - val_dice_coef: 0.8485 - lr: 5.0000e-04
Epoch 77/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.2241 - dice_coef: 0.8705
Epoch 00077: val_loss did not improve from 0.20230
2109/2109 [==============================] - 202s 96ms/step - loss: 0.2241 - dice_coef: 0.8706 - val_loss: 0.2191 - val_dice_coef: 0.8393 - lr: 5.0000e-04
Epoch 78/300
2108/2109 [============================>.] - ETA: 0s - loss: 0.226

Epoch 100/300
 159/2109 [=>............................] - ETA: 2:56 - loss: 0.1802 - dice_coef: 0.8720